<a href="https://colab.research.google.com/github/yasyamauchi/education/blob/main/decision_tree_BME2024_report3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 決定木 (けっていぎ，decision tree)  
  
  2024新版  
[Google Machine Learning: 上級コース: デシジョンフォレスト](https://developers.google.com/machine-learning/decision-forests/practice?hl=ja)より  

YDF（Yggdrasil）を使用する．

In [ ]:
!pip install ydf -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 34.9 MB/s eta 0:00:00


In [ ]:
import ydf
import numpy as np
import pandas as pd

## データの読み込み  
  
  この例では"Palmer Penguins"データセットを使用する．  
### データの中身  
3種類のペンギン(合計344羽)  
* チンストラップ(Chinstrap)
* ジェンツー(Gentoo)
* アデリー(Adelie)  
  
の，次のデータを使用する．  
* 生息地 (island)
* くちばしの長さ (bill_length)
* くちばしの上下幅 (bill_depth)
* 羽の長さ (flipper_length)
* 体重 (body_mass)
* 性別 (sex)
* 誕生年？ (year)  
  
最初の3羽のデータを表示する．

In [ ]:
#path = "https://raw.githubusercontent.com/google/yggdrasil-decision-forests/main/yggdrasil_decision_forests/test_data/dataset/adult_train.csv"
#dataset = pd.read_csv(path)
#label = "income"

# Display the first 3 examples.
dataset.head(10)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,44,Private,228057,7th-8th,4,Married-civ-spouse,Machine-op-inspct,Wife,White,Female,0,0,40,Dominican-Republic,<=50K
1,20,Private,299047,Some-college,10,Never-married,Other-service,Not-in-family,White,Female,0,0,20,United-States,<=50K
2,40,Private,342164,HS-grad,9,Separated,Adm-clerical,Unmarried,White,Female,0,0,37,United-States,<=50K
3,30,Private,361742,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,<=50K
4,67,Self-emp-inc,171564,HS-grad,9,Married-civ-spouse,Prof-specialty,Wife,White,Female,20051,0,30,England,>50K
5,18,Private,318082,Some-college,10,Never-married,Sales,Own-child,White,Male,0,0,35,United-States,<=50K
6,51,Private,200618,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,<=50K
7,32,Private,295117,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,40,England,>50K
8,33,Private,355996,10th,6,Never-married,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
9,46,Private,265097,HS-grad,9,Never-married,Handlers-cleaners,Other-relative,White,Male,0,0,5,United-States,<=50K


## 訓練データと検証データに分ける  
  
344のデータの約1割を検証データに，残りを訓練データに分ける．ランダムに分けるので実行するたびに結果は異なる．

In [ ]:
# Use the ~20% of the examples as the testing set
# and the remaining ~80% of the examples as the training set.
np.random.seed(1)
is_test = np.random.rand(len(dataset)) < 0.2

train_dataset = dataset[~is_test]
test_dataset = dataset[is_test]

print("Training examples: ", len(train_dataset))
# >> Training examples: 272

print("Testing examples: ", len(test_dataset))
# >> Testing examples: 72

Training examples:  18254
Testing examples:  4538


## モデルの学習  
  
デフォルト値で行う．学習はすぐ終わる．

In [ ]:
model = ydf.CartLearner(label=label).train(train_dataset)

Train model on 18254 examples
Model trained in 0:00:00.079126


## モデルの評価
  
  まず決定木を表示する．見方は次の通り．  
* 左から右に見る
* 赤(0)がアデリー，青(1)がジェンツー，緑(2)がチンストラップ
* 分類された結果が棒グラフで表示されており，その上に条件が示されている．例えば最初の**flipper_length_mm >= 207.500**は**「羽の長さが207.5mm以上かどうか」**を意味する．Yesが上でNoが下．
* マウスを用いると，詳細な数字が表示される．

In [ ]:
model.plot_tree()

精度を計算する．訓練データ93%・検証データで92％程度となる．実際の精度は実行するたびにわずかに異なる(理由は考えること)．

In [ ]:
train_evaluation = model.evaluate(train_dataset)
print("train accuracy:", train_evaluation.accuracy)
# >> train accuracy:  0.9338

test_evaluation = model.evaluate(test_dataset)
print("test accuracy:", test_evaluation.accuracy)
# >> test accuracy:  0.9167

train accuracy: 0.8653993645228443
test accuracy: 0.8565447333627149


木の深さを最大2とする

In [ ]:
model = ydf.CartLearner(label=label, max_depth=3).train(train_dataset)
print(model.evaluate(test_dataset).accuracy)
# >> 0.97222

Train model on 18254 examples
Model trained in 0:00:00.016421
0.8338475099162627


In [ ]:
model.plot_tree()

ランダムフォレスト法【後述】を適用する  
精度はかなり改善するはず

In [ ]:
model = ydf.RandomForestLearner(label=label).train(train_dataset)
model.plot_tree()
print("Test accuracy: ", model.evaluate(test_dataset).accuracy)
# >> Test accuracy: 0.986111

Train model on 18254 examples
Model trained in 0:00:09.985958
Test accuracy:  0.8616130453944469


In [ ]:
model.plot_tree()